In [1]:
from sys import exit
import numpy as np
import pandas as pd
from os.path import dirname, join, isfile, isdir
from os import makedirs
from datetime import datetime, timedelta
from hashlib import sha256

from bulletin.data.tb_pacientes import TbPacientes
from bulletin.data.casos_confirmados import CasosConfirmados
from bulletin.commom.utils import Timer
from bulletin.commom import static
from bulletin.commom.normalize import *

pd.set_option('display.max_columns', None)

output = join("output","correcoes","casos_confirmados")

if not isdir(output):
    makedirs(output)

timer = Timer()

In [2]:
timer.reset()
casos_confirmados = CasosConfirmados()
# casos_confirmados.update()
timer.stop()

0.5618970394134521

In [3]:
casos = casos_confirmados.get_casos()
casos.shape

(329782, 16)

In [4]:
casos = casos_confirmados.get_casos()

casos['ibge7'] = casos['ibge7'].apply(lambda x: str(x) if x != 99 else None)
casos['ibge'] = casos['ibge7'].apply(normalize_igbe)

casos_sem_ibge = casos.loc[casos['ibge'].isnull()].copy()
casos_sem_ibge = casos_sem_ibge.drop(columns=['ibge'])
casos_sem_ibge['mun_hash'] = casos_sem_ibge['mun_resid'].apply(normalize_hash)


municipios = static.municipios.copy()

casos_com_ibge = casos.loc[casos['ibge'].notnull()].copy()
casos_com_ibge = pd.merge(left=casos_com_ibge, right=municipios, how='left', on='ibge')

municipios = municipios.loc[municipios['uf']!='PR']
municipios['mun_hash'] = municipios['municipio_sesa'].apply(lambda x: normalize_hash(normalize_text(x)))
municipios_hash = municipios.drop_duplicates('mun_hash')

casos_sem_ibge = pd.merge(left=casos_sem_ibge, right=municipios_hash, how='left', on='mun_hash')
casos_sem_ibge = casos_sem_ibge.drop(columns=['mun_hash'])

casos_com_ibge = casos_com_ibge.append(casos_sem_ibge.loc[casos_sem_ibge['ibge'].notnull()], ignore_index=True).sort_values('ordem')

casos_sem_ibge = casos_sem_ibge.loc[casos_sem_ibge['ibge'].isnull()].copy()
casos_sem_ibge = casos_sem_ibge.drop(columns=["cod_uf", "uf", "estado", "ibge", "municipio_sesa", "municipio_ibge"])
casos_sem_ibge['mun_hash'] = casos_sem_ibge['mun_resid'].apply(normalize_hash)

municipios['mun_hash'] = municipios['municipio_ibge'].apply(lambda x: normalize_hash(normalize_text(x)))
municipios_hash = municipios.drop_duplicates('mun_hash')

casos_sem_ibge = pd.merge(left=casos_sem_ibge, right=municipios_hash, how='left', on='mun_hash')
casos = casos_com_ibge.append(casos_sem_ibge, ignore_index=True).sort_values('ordem')

casos['ibge'] = casos['ibge'].fillna('99')
casos['ibge'] = casos['ibge'].apply(lambda x: str(x).zfill(2) if x != '99' else None)

casos = casos[['ordem','ibge7','ibge','nome','sexo','idade','mun_resid','uf_resid','municipio_sesa','municipio_ibge','uf','mun_atend','rs','laboratorio','dt_diag','comunicacao','is','hash','hash_less','hash_more']]

In [5]:
casos.loc[casos['ibge'].isnull()].to_excel(join(output,'casos_sem_ibge.xlsx'))
# casos.loc[casos['ibge7'].isnull()].to_excel(join(output,'casos_sem_ibge7.xlsx'))
casos.loc[(casos['uf_resid'] == 'PR') & (casos['uf'].notnull()) & (casos['uf'] != 'PR'), 'uf_resid'] = casos.loc[(casos['uf_resid'] == 'PR') & (casos['uf'].notnull()) & (casos['uf'] != 'PR'), 'uf']
casos.loc[(casos['uf_resid'] == 'PR') & (casos['uf'].isnull()), 'uf_resid'] = 'ERRO'
casos.loc[casos['uf']!='PR', 'mun_resid'] = casos.loc[casos['uf']!='PR', 'mun_resid'] + '/' + casos.loc[casos['uf']!='PR', 'uf_resid']
casos.loc[casos['uf']!='PR', ['ordem','mun_resid']]
casos[['ordem','mun_resid']].to_excel(join(output,"municipios_normalizados.xlsx"))


KeyError: 'mun_nome'

In [8]:
obitos = casos_confirmados.get_obitos()
obitos.shape

(6764, 7)

In [9]:
obitos.loc[obitos.duplicated('hash')].append(obitos.loc[obitos['hash'].isin(obitos['hash_less'])],ignore_index=True).append(obitos.loc[obitos['hash'].isin(obitos['hash_more'])],ignore_index=True).to_excel(join(output,'obitos_duplicados.xlsx'))

In [10]:
# obitos_nao_em_casos = obitos.loc[~obitos['hash'].isin(casos['hash'])]
# obitos_nao_em_casos.to_excel(join(output,'obitos_nao_em_casos.xlsx'))

# len(obitos_nao_em_casos)

# casos_idade_errada_less = obitos_nao_em_casos.loc[obitos['hash_less'].isin(casos['hash']), 'hash_less'].values.tolist()
# casos_idade_errada_more = obitos_nao_em_casos.loc[obitos['hash_more'].isin(casos['hash']), 'hash_more'].values.tolist()

# casos.loc[casos['hash'].isin(casos_idade_errada_less), 'idade'] -=1

# casos.loc[casos['hash'].isin(casos_idade_errada_less), 'hash'] = casos.loc[casos['hash'].isin(casos_idade_errada_less)].apply(lambda row: sha256(str.encode(normalize_hash(row['nome'])+str(row['idade'])+normalize_hash(row['mun_resid']))).hexdigest(), axis=1)
# casos.loc[casos['hash'].isin(casos_idade_errada_less), 'hash_less'] = casos.loc[casos['hash'].isin(casos_idade_errada_less)].apply(lambda row: sha256(str.encode(normalize_hash(row['nome'])+str(row['idade']-1)+normalize_hash(row['mun_resid']))).hexdigest(), axis=1)
# casos.loc[casos['hash'].isin(casos_idade_errada_less), 'hash_more'] = casos.loc[casos['hash'].isin(casos_idade_errada_less)].apply(lambda row: sha256(str.encode(normalize_hash(row['nome'])+str(row['idade']+1)+normalize_hash(row['mun_resid']))).hexdigest(), axis=1)


# casos.loc[casos['hash'].isin(casos_idade_errada_more), 'idade'] +=1
# casos.loc[casos['hash'].isin(casos_idade_errada_more), 'hash'] = casos.loc[casos['hash'].isin(casos_idade_errada_more)].apply(lambda row: sha256(str.encode(normalize_hash(row['nome'])+str(row['idade'])+normalize_hash(row['mun_resid']))).hexdigest(), axis=1)
# casos.loc[casos['hash'].isin(casos_idade_errada_more), 'hash_less'] = casos.loc[casos['hash'].isin(casos_idade_errada_more)].apply(lambda row: sha256(str.encode(normalize_hash(row['nome'])+str(row['idade']-1)+normalize_hash(row['mun_resid']))).hexdigest(), axis=1)
# casos.loc[casos['hash'].isin(casos_idade_errada_more), 'hash_more'] = casos.loc[casos['hash'].isin(casos_idade_errada_more)].apply(lambda row: sha256(str.encode(normalize_hash(row['nome'])+str(row['idade']+1)+normalize_hash(row['mun_resid']))).hexdigest(), axis=1)

# obitos.loc[~obitos['hash'].isin(casos['hash'])]

In [11]:
exit()
duplicados = casos.loc[casos.duplicated('hash')].index.tolist()
duplicados_less = casos.loc[casos['hash'].isin(casos['hash_less'])].index.tolist()
duplicados_more = casos.loc[casos['hash'].isin(casos['hash_more'])].index.tolist()
dropar = set(duplicados + duplicados_less + duplicados_more)
casos.loc[dropar].to_excel('casos_duplicados.xlsx')
print(len(dropar))
casos = casos.drop(index=dropar)
len(casos)

SystemExit: 

In [ ]:
duplicados = casos.loc[casos.duplicated('hash')].index.tolist()
duplicados_less = casos.loc[casos['hash'].isin(casos['hash_less'])].index.tolist()
duplicados_more = casos.loc[casos['hash'].isin(casos['hash_more'])].index.tolist()
dropar = set(duplicados + duplicados_less + duplicados_more)
casos.loc[dropar].to_excel('obitos_duplicados_casos_confirmados.xlsx')
len(dropar)

In [ ]:
writer = pd.ExcelWriter("novos_casos_confirmados.xlsx",
                        engine='xlsxwriter',
                        datetime_format='dd/mm/yyyy',
                        date_format='dd/mm/yyyy')

casos.sort_values('nome').to_excel(writer,index=None)

writer.save()
writer.close()